## Flask-Limiter:
extension for Flask that provides rate-limiting capabilities. Rate limiting is an essential feature to prevent abuse of your application by limiting the number of requests a user can make to certain endpoints in a given period of time.

In [ ]:
# Installation
!pip install flask-limiter

In [ ]:
from flask_limiter import Limiter

limiter = Limiter(app)

### Fixed Vs Sliding Window

<table border="1" cellpadding="10" cellspacing="0">
  <thead>
    <tr>
      <th>Feature</th>
      <th>Fixed Window</th>
      <th>Sliding Window</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Definition</td>
      <td>Rate limit resets after a fixed time interval (e.g., every minute, hour).</td>
      <td>Rate limit is based on a rolling window of time, checking the most recent requests over the last X seconds or minutes.</td>
    </tr>
    <tr>
      <td>Time Window</td>
      <td>Fixed period (e.g., every 1 minute or 1 hour).</td>
      <td>Rolling period based on recent requests (e.g., last 60 seconds, last 5 minutes).</td>
    </tr>
    <tr>
      <td>Request Tracking</td>
      <td>Requests are tracked for a specific time period and reset at the end of that period.</td>
      <td>Requests are tracked based on the timestamp of each request, with the window moving forward as time progresses.</td>
    </tr>
    <tr>
      <td>Rate Limit Reset</td>
      <td>Resets after the fixed period ends, regardless of when the requests were made.</td>
      <td>The window continuously slides forward, adjusting the rate limit dynamically based on the most recent requests.</td>
    </tr>
    <tr>
      <td>Use Case</td>
      <td>Simple rate limiting where the user can make a fixed number of requests within a specific time period.</td>
      <td>More dynamic rate limiting, providing smoother and fairer limits for users by considering recent activity.</td>
    </tr>
    <tr>
      <td>Edge Cases</td>
      <td>A user who hits the limit just before the window resets may be at a disadvantage, needing to wait for the reset.</td>
      <td>More fair in cases where users are close to the limit but still within a rolling window.</td>
    </tr>
  </tbody>
</table>


#### Sliding Window Example
- **Limit**: The @limiter.limit("5 per minute", ...) decorator indicates that the user is allowed to make up to 5 requests within a 1-minute sliding window.
- **Sliding Window**: If a user makes 5 requests within the last 60 seconds, further requests will be blocked until the time of one of the requests moves past the 60-second window.

### The @limiter.limit Decorator
allows you to restrict the number of requests that a specific route can receive within a given time window.

In [ ]:
@limiter.limit("X per Y")

- **X**: Maximum number of requests allowed.
- **Y**: Time window (e.g., per minute, per hour).

#### Time Units you can use:
##### **Seconds**:
Example: "5 per second" or "10 per 30 seconds"

##### **Minutes**:
Example: "2 per minute" or "1 per 5 minutes"

##### **Hours**:
Example: "100 per hour" or "10 per 30 minutes" (combined with minutes)

##### **Days**:
Example: "1 per day" or "7 per week" (using the day as a reference)

### Flask-Limiter Configuration

- **DEFAULT_LIMITS**: Global default rate limits for all routes in the application.
- **STORAGE_URI**: The URI for the rate limit storage backend (e.g., Redis, Memcached).
- **KEY_FUNC**: Custom function to define how rate limit keys are generated.


In [ ]:
app.config['LIMITS'] = ["10 per hour"]
app.config['STORAGE_URI'] = "redis://localhost:6379"
limiter = Limiter(app)

### Global Rate Limits
You can set a global rate limit that applies to all routes in the application.

In [ ]:
app.config['LIMITS'] = ["10 per hour"]  # 10 requests per hour for all routes
limiter = Limiter(app)

### Dynamic Rate Limits
You can apply dynamic rate limits based on request data, such as the user ID, IP address, or any custom logic.

In [ ]:
from flask import g

@app.route("/dynamic_limited")
@limiter.limit(lambda: "10 per hour" if g.user.is_authenticated else "3 per hour")
def dynamic_limited_view():
    return "This view has a dynamic rate limit based on authentication status."

In this example:
- Authenticated users are limited to 10 requests per hour.
- Non-authenticated users are limited to 3 requests per hour.

### Rate Limit Exemption (@limiter.exempt)
some routes that should not be subject to rate limiting, such as health check routes or special API endpoints. You can use the @limiter.exempt decorator to exclude specific routes from rate limiting.

In [ ]:
@app.route("/health_check")
@limiter.exempt
def health_check():
    return "OK"

### Custom Error Messages
allows you to customize the error message that is returned when a user exceeds the rate limit.

In [ ]:
@limiter.limit("2 per minute", error_message="You have exceeded the request limit. Please try again later.")

### IP-based Rate Limiting
You can set up rate limits based on the user's IP address using the key_func parameter. This is useful for preventing abuse from a specific IP.

In [ ]:
@limiter.limit("5 per minute", key_func=lambda: request.remote_addr)

### Managing Rate Limit Counters
provides a backend for storing rate limit counters. By default, it uses in-memory storage. However, you can use other backends like Redis or Memcached for production environments

In [ ]:
# Redis Installation
! pip install redis

In [ ]:
from flask_limiter import Limiter
from redis import Redis

app = Flask(__name__)
limiter = Limiter(app, storage_uri="redis://localhost:6379")

Rate limits are stored in a Redis database.
This is more scalable for production applications compared to using in-memory storage.

### Example

In [ ]:
from flask import Flask
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

app = Flask(_name_)
limiter = Limiter(app, default_limits=["5 per minute"],key_func=get_remote_address)

@app.route('/')
@limiter.limit("2 per minute")
def index():
    return 'This route is limited to 2 requests per minute'


**get_remote_address** specifys that each ip address has this requests limit "5 per minute".

**Explanation:(suppose in index route we made 2 requests in first 10 ses)**<br>
The Limiter will track the number of requests made to that route within the specified time frame (1 minute in this case). Since you've already hit the limit of 2 requests, any further requests within that minute will be blocked.<br>
the server will respond with a 429 Too Many Requests status code.